<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session8_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 8 (Syntactic Parser) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [ ]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/


Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 393 (delta 139), reused 89 (delta 53), pack-reused 190
Receiving objects: 100% (393/393), 327.88 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scripts.text_preprocessing import TextPreprocessing
from complementary_material.textserver import TextServer
import nltk
from nltk import CFG, ChartParser
!pip install svgling
import svgling


## TODO

- Consider the following sentence: **Lazy cats play with mice.**

- Expand the grammar of the example related to non-probabilistic chart parsers in order to subsume this new sentence.

- Perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.

- For each one of them, provide the resulting tree, the number of edges and the list of explored edges.

- Which parser is the most efficient for parsing the sentence?

- Which edges are filtered out by each parser and why?

In [55]:
phrase="Lazy cats play with mice."
test_sentence="Lazy cats and mice."

In [62]:
tp = TextPreprocessing()

clean_sentence=tp.clean_sentence(phrase, signs=True)

TypeError: ignored

In [61]:
clean_sentence

['lazy', 'cats', 'play', 'with', 'mice.']

In [ ]:
def sentence_to_cfg(sentence):
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    productions = []

    for i in range(len(pos_tags)):
        productions.append(f"Word_{i} -> '{words[i]}'")
        productions.append(f"POS_{i} -> '{pos_tags[i][1]}'")

    for i in range(len(pos_tags) - 1):
        productions.append(f"Phrase_{i} -> Word_{i} | POS_{i}")
        productions.append(f"Sentence -> Phrase_{i} Phrase_{i+1}")

    cfg_string = "\n".join(productions)

    return CFG.fromstring(cfg_string)

# Example sentence
sentence = "The cat chased the mouse"

# Convert sentence to CFG
cfg = sentence_to_cfg(sentence)

# Print the CFG
print(cfg)

In [ ]:
grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Adj N | N
    VP -> V NP
    Adj -> 'Lazy'
    N -> 'cats' | 'mice'
    V -> 'play' | 'with'
""") #added manually, shall we create a function to obtain it?

  #Do we need to first pass everything to chomsky normal form?


parser = ChartParser(grammar, trace=1)
parse = parser.chart_parse(clean_test_sentence)




|.  small  .   cats  .   and   .   mice  .|
|[---------]         .         .         .| [0:1] 'small'
|.         [---------]         .         .| [1:2] 'cats'
|.         .         [---------]         .| [2:3] 'and'
|.         .         .         [---------]| [3:4] 'mice'
|[---------]         .         .         .| [0:1] JJ -> 'small' *
|[--------->         .         .         .| [0:1] NP -> JJ * NNS
|.         [---------]         .         .| [1:2] NNS -> 'cats' *
|.         [---------]         .         .| [1:2] NP -> NNS *
|.         [--------->         .         .| [1:2] NNS -> NNS * CC NNS
|[-------------------]         .         .| [0:2] NP -> JJ NNS *
|[------------------->         .         .| [0:2] NP -> NP * CC NP
|.         [--------->         .         .| [1:2] NP -> NP * CC NP
|.         .         [---------]         .| [2:3] CC -> 'and' *
|.         [------------------->         .| [1:3] NNS -> NNS CC * NNS
|[----------------------------->         .| [0:3] NP -> NP CC * NP

In [ ]:
print(f'Total edges: {parse.num_edges()}\n')
parse.edges()
#Do we need to add the main sentence?  S -> 'small cats and mice'

Total edges: 28



[[Edge: [0:1] 'small'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'and'],
 [Edge: [3:4] 'mice'],
 [Edge: [0:1] JJ -> 'small' *],
 [Edge: [0:1] NP -> JJ * NNS],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [0:2] NP -> JJ NNS *],
 [Edge: [0:2] NP -> NP * CC NP],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [2:3] CC -> 'and' *],
 [Edge: [1:3] NNS -> NNS CC * NNS],
 [Edge: [0:3] NP -> NP CC * NP],
 [Edge: [1:3] NP -> NP CC * NP],
 [Edge: [3:4] NNS -> 'mice' *],
 [Edge: [3:4] NP -> NNS *],
 [Edge: [3:4] NNS -> NNS * CC NNS],
 [Edge: [1:4] NNS -> NNS CC NNS *],
 [Edge: [1:4] NP -> NNS *],
 [Edge: [1:4] NNS -> NNS * CC NNS],
 [Edge: [0:4] NP -> JJ NNS *],
 [Edge: [0:4] NP -> NP * CC NP],
 [Edge: [1:4] NP -> NP * CC NP],
 [Edge: [3:4] NP -> NP * CC NP],
 [Edge: [0:4] NP -> NP CC NP *],
 [Edge: [1:4] NP -> NP CC NP *]]

In [ ]:
ts = list(parse)
'{num} trees.'.format(num=len(ts))

'28 trees.'

In [ ]:
print(ts[0])

[0:1] 'small'


In [ ]:
ts[0]

[Edge: [0:1] 'small']

In [ ]:
ts[1]

[Edge: [1:2] 'cats']

In [ ]:
from nltk import TopDownChartParser
parser = nltk.TopDownChartParser(grammar)
parse = parser.parse(sent)

In [51]:
import nltk
from nltk import CFG

def sentence_to_cfg(sentence):
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    productions = []

    for i in range(len(pos_tags)):
        productions.append(f"Word_{i} -> '{words[i]}'")
        productions.append(f"POS_{i} -> '{pos_tags[i][1]}'")

    for i in range(len(pos_tags) - 1):
        productions.append(f"Phrase_{i} -> Word_{i} | POS_{i}")
        productions.append(f"Sentence -> Phrase_{i} Phrase_{i+1}")

    cfg_string = "\n".join(productions)

    return CFG.fromstring(cfg_string)

# Example sentence
sentence = "The cat chased the mouse"

# Convert sentence to CFG
cfg = sentence_to_cfg(sentence)

# Print the CFG
print(cfg)

Grammar with 22 productions (start state = Word_0)
    Word_0 -> 'The'
    POS_0 -> 'DT'
    Word_1 -> 'cat'
    POS_1 -> 'NN'
    Word_2 -> 'chased'
    POS_2 -> 'VBD'
    Word_3 -> 'the'
    POS_3 -> 'DT'
    Word_4 -> 'mouse'
    POS_4 -> 'NN'
    Phrase_0 -> Word_0
    Phrase_0 -> POS_0
    Sentence -> Phrase_0 Phrase_1
    Phrase_1 -> Word_1
    Phrase_1 -> POS_1
    Sentence -> Phrase_1 Phrase_2
    Phrase_2 -> Word_2
    Phrase_2 -> POS_2
    Sentence -> Phrase_2 Phrase_3
    Phrase_3 -> Word_3
    Phrase_3 -> POS_3
    Sentence -> Phrase_3 Phrase_4
